# Homework #5 Deploy Model in GCP (File 2 of 2)
### File 1: Notebook to create the model and save the model on GCP AI platform
### File 2: Notebook that is calling the service on GCP AI platform

---

# Import Libraries

In [0]:
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import re
import spacy
from sklearn.preprocessing import LabelEncoder

In [0]:
import googleapiclient.discovery

# Load Secure Key

In [0]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "utopian-rush-255612-b73345de1c22.json"

# Setup Environments and Functions

In [0]:
project_id = "utopian-rush-255612"
model_id = "my_sentiment_analysis"
model_path = "projects/{}/models/{}".format(project_id,model_id)
ml_resource = googleapiclient.discovery.build("ml", "v1").projects()
output_name = 'dense_1'

In [0]:
def predict(X):
    input_data_json = {"signature_name": "serving_default", "instances": X.tolist()}
    request = ml_resource.predict(name=model_path, body=input_data_json)
    response = request.execute()
    if "error" in response:
        raise RuntimeError(response["error"])
    return np.array([pred[output_name] for pred in response["predictions"]])
    # return response["predictions"]

# Generate tweets

In [0]:
# Create input tweets for testing the service 
tweets = {'tweets':['The seat was comfortable! and the bathroom was clean and food was great...',
                    'The flight attendant was nice ### but the temperature was cold ---',
                    'The movie was not funny at all and the drinks were very bad...',
                    'huh uhgyrt jjkoj . g red jgh kh kj k',
                    'I was told that the flight would be bad but actually it was very good, thanks US Ailine',
                    'I was told that the flight would be good but actually it was very bad, thanks US Ailine'
                   ],
          'sentiments':['positive',
                       'neutral',
                       'negative',
                       'neutral',
                       'positive',
                       'negative'
                      ]
         } 
  
# Create DataFrame 
df = pd.DataFrame(tweets) 
  
# Print the output. 
df 

,sentiments,tweets
0,positive,The seat was comfortable! and the bathroom was...
1,neutral,The flight attendant was nice ### but the temp...
2,negative,The movie was not funny at all and the drinks ...
3,neutral,huh uhgyrt jjkoj . g red jgh kh kj k
4,positive,I was told that the flight would be bad but ac...
5,negative,I was told that the flight would be good but a...


# Data Preprocessing

In [0]:
sentiments = df["sentiments"].copy()
label_encoder = LabelEncoder()
sentiments = label_encoder.fit_transform(sentiments)
sentiments = keras.utils.to_categorical(sentiments)
sentiments[:5]

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [0]:
# remove URL's from tweets
df['clean_tweets'] = df['tweets'].apply(lambda x: re.sub(r'http\S+', '', x))

In [0]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

df['clean_tweets'] = df['clean_tweets'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
df['clean_tweets'] = df['clean_tweets'].str.lower()

# remove numbers
df['clean_tweets'] = df['clean_tweets'].str.replace("[0-9]", " ")

# remove whitespaces
df['clean_tweets'] = df['clean_tweets'].apply(lambda x:' '.join(x.split()))

### Lemmatize (Normalize) Tweets

In [0]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [0]:
df['clean_tweets'] = lemmatization(df['clean_tweets'])

In [0]:
df

,sentiments,tweets,clean_tweets
0,positive,The seat was comfortable! and the bathroom was...,the seat be comfortable and the bathroom be cl...
1,neutral,The flight attendant was nice ### but the temp...,the flight attendant be nice but the temperatu...
2,negative,The movie was not funny at all and the drinks ...,the movie be not funny at all and the drink be...
3,neutral,huh uhgyrt jjkoj . g red jgh kh kj k,huh uhgyrt jjkoj . g red jgh kh kj k
4,positive,I was told that the flight would be bad but ac...,i be tell that the flight would be bad but act...
5,negative,I was told that the flight would be good but a...,i be tell that the flight would be good but ac...


In [0]:
X_input = df["clean_tweets"].values

In [0]:
y_pred = model.predict(X_input)

NameError: name 'model' is not defined

# Call the service my_sentiment_analysis with the input data (tweets)

In [0]:
y_pred = predict(X_input)

# Show Predictions

In [0]:
print ("Predictions" + "\n" +str(y_pred.round(1)))

Predictions
[[0.1 0.  0.9]
 [0.4 0.  0.6]
 [0.7 0.  0.2]
 [0.  0.9 0.1]
 [0.4 0.1 0.6]
 [0.4 0.1 0.6]]


In [0]:
print ("Actual sentiments" + "\n" +str(sentiments))

Actual sentiments
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


<img src="GCP Service Monitoring.png">